In [74]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower().split()
    if "television" in new_award_names[i]:
        new_award_names[i].append("tv")
new_award_names

[['best', 'motion', 'picture', 'drama'],
 ['best', 'motion', 'picture', 'musical', 'comedy'],
 ['best', 'motion', 'picture', 'foreign', 'language'],
 ['best', 'motion', 'picture', 'animated'],
 ['best', 'director', 'motion', 'picture'],
 ['best', 'actor', 'motion', 'picture', 'drama'],
 ['best', 'actor', 'motion', 'picture', 'musical', 'comedy'],
 ['best', 'actress', 'motion', 'picture', 'drama'],
 ['best', 'actress', 'motion', 'picture', 'musical', 'comedy'],
 ['best', 'supporting', 'actor', 'motion', 'picture'],
 ['best', 'supporting', 'actress', 'motion', 'picture'],
 ['best', 'screenplay', 'motion', 'picture'],
 ['best', 'original', 'score', 'motion', 'picture'],
 ['best', 'original', 'song', 'motion', 'picture'],
 ['best', 'television', 'series', 'drama', 'tv'],
 ['best', 'television', 'series', 'musical', 'comedy', 'tv'],
 ['best', 'miniseries', 'motion', 'picture', 'television', 'tv'],
 ['best', 'actor', 'television', 'series', 'drama', 'tv'],
 ['best', 'actor', 'television', 's

In [75]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [76]:
df = pd.read_json('../gg2013.json')
df = df['text']
df = df[~df.str.contains('think|thinking|should|maybe', case = False)]
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
wins_df = df[df.str.contains('won best|wins best|goes to', case = False)]

wins_df

3490      RT @yeahitsaline_: And the   to "Best Twitter ...
4511      RT @EricaDeLong:  #EricaRedCarpet Dustin Hoffm...
7462      And  goes tooooooooo... Oğlum kara kışta böyle...
9194      Biggest zinger so far at the  goes to Amy Poeh...
9730      Watching the  ! I say silver linings wins best...
                                ...                        
174473    RT @playbill: "Les Miserables" wins Best Motio...
174567    RT @nerdywonka: Ben Affleck won Best Director ...
174576    Best Actor, Drama:   For Film Goes To Daniel D...
174578    Best Actor, Drama:   For Film Goes To Daniel D...
174580    Best Actor, Drama:   For Film Goes To Daniel D...
Name: text, Length: 3806, dtype: object

In [77]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [78]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
#pattern2 = [{"TEXT": "goes to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}]

In [79]:
matcher = Matcher(nlp.vocab)
matcher.add("Winner", [pattern1])

In [80]:
winners = {}
winners = defaultdict(lambda: 1, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        winners[str(span).strip(punctuation).strip()] = winners[str(span).strip(punctuation).strip()] +1

In [81]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

[('Adele wins Best Original Song', 29), ('Christoph Waltz Wins Best Supporting Actor', 24), ('Christoph Waltz wins Best Supporting Actor', 13), ('Daniel Day-Lewis wins Best Actor Motion Picture - Drama', 13), ('Adele won Best Original Song', 6), ('Christoph Waltz won Best Supporting Actor', 5), ('Ben Affleck wins Best Director - Motion Picture', 5), ('Les Miserables wins Best Motion Picture', 5), ('Adele wins Best Original Song Motion Picture', 4), ('Jennifer Lawrence wins Best Actress Motion Picture Comedy', 4), ('Adele Wins Best Original Song', 4), ('Damian Lewis wins Best Actor TV Series', 3), ('Don Cheadle Wins Best TV Actor - Comedy', 3), ('Mychael Danna wins Best Original Score', 3), ('Ben Affleck wins Best Director prize', 3), ('Christoph Waltz wins Best Supporting Actor - Motion Picture', 2), ('Christoph Waltz WINS Best Supporting Actor', 2), ('Yayyyy Adele wins Best Original Song', 2), ('Christoph Waltz Wins Best Supportin Actor', 2), ('Glad Adele won Best Original Song', 2), 

In [132]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = split[index:]

    name_count = [0]*len(new_award_names)
    for i in range(len(new_award_names)):
        match_length = len(set(new_award_names[i]).intersection(award_phrase))
        name_count[i] = match_length

    award_index = np.argmax(name_count)
    # WHAT ABOUT TIE? Neither? Or Both?
    #print(award_index)

    if name_count[award_index] > 1:       
        if "wins" in split:
            person_index = split.index("wins")
        if "won" in split:
            person_index = split.index("won")
        person = split[:index-1]
        person = " ".join(person)
        if person not in award_list[award_index]:
            award_list[award_index][person] = 1
        else:
            award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

[{'les miserables': 1, 'les mis': 1, 'jennifer lawrence': 1}, {'les misèrables': 1, 'les misérables': 1}, {}, {}, {'ben affleck': 3, 'quentin tarantino': 1}, {'daniel day-lewis': 3, 'christoph waltz': 1, 'hugh jackman': 3, 'daniel day lewis': 2}, {'don cheadle': 1}, {'jennifer lawrence': 2, 'jessica chastain': 2}, {'jennifer lawrence': 1}, {'christoph waltz': 5, 'ed harris': 1}, {}, {'quentin tarantino': 1}, {'mychael danna': 1}, {'adele': 5, 'yayyyy adele': 1, 'glad adele': 1}, {}, {}, {}, {'damian lewis': 1}, {'don cheadle': 1}, {'kevin costner': 1}, {'claire danes': 4, 'lena dunham': 1}, {}, {}, {}, {}]


In [136]:
import operator
i = 0
awards = {}
for award in award_list:
    if award != {}:
        awards[i] = max(award.items(), key=operator.itemgetter(1))[0]
    else:
        awards[i] = ''
    i+=1
print(awards)

{0: 'les miserables', 1: 'les misèrables', 2: '', 3: '', 4: 'ben affleck', 5: 'daniel day-lewis', 6: 'don cheadle', 7: 'jennifer lawrence', 8: 'jennifer lawrence', 9: 'christoph waltz', 10: '', 11: 'quentin tarantino', 12: 'mychael danna', 13: 'adele', 14: '', 15: '', 16: '', 17: 'damian lewis', 18: 'don cheadle', 19: 'kevin costner', 20: 'claire danes', 21: '', 22: '', 23: '', 24: ''}
